# Phase 5: Diagnosing Outputs from Running XML-sets and Generate Report

This notebook  attempts to do what the software [Tracer](https://beast.community/tracer) does and make some improvements. 

## Instructions

Code cells of this Jupyter notebook should be run sequentially via shift+enter. Several cells will produce widgets that allow you to make various selections to select for MCMC chains that have converged. Once you have made that selection left-click on the cell below and press shift+enter.

## Suggested Reading

Up to and including "**x% HPD interval**" of:

Drummond, Alexei J., and Bouckaert, Remco R. ‘Ch 10: Posterior Analysis and Post Processing.’ In Bayesian Evolutionary Analyses with BEAST. Cambridge University Press, 2015. https://www.cambridge.org/core/books/bayesian-evolutionary-analysis-with-beast/81F5894F05E87F13C688ADB00178EE00.

The authors have been kind enough to make a draft copy of the book available at http://alexeidrummond.org/assets/publications/2015-drummond-bayesian.pdf.

## Parameters
<details>
    <summary>Click To See A Decription of Parameters</summary>
        <pre>
            <code>
beast_outputs: str
    A valid path containing output `.log` and `.tree` files from running BEAST 2

report_template: str
    Name of a valid report template to use to generate report.

kernel: str, default 'beast_pype'
    Name of Jupyter python kernel to use when running report template notebooks.

  </code>
</pre>

In [ ]:
report_template = None
xml_set_label = 'xml_set'
beast_xml_path = None
kernel_name = 'beast_pype'

Import necessary packages.

In [ ]:
from copy import deepcopy
import yaml
import papermill as pm
from beast_pype.nb_utils import execute_notebook
import pandas as pd
from beast_pype.diagnostics.mcmc import BEASTDiag
from beast_pype.report_gen import add_unreported_outputs, gen_mcc_notebook
from beast_pype.diagnostics.runtime import get_beast_runtimes, get_slurm_job_stats
import pandas as pd
import warnings
import os
import importlib.resources as importlib_resources
# stop annoying matplotlib warnings
warnings.filterwarnings("ignore", module="matplotlib\*")

In [ ]:
workflow_modules = importlib_resources.path('beast_pype', 'workflow_modules')
save_dir=os.getcwd()

Get
pipeline
run
info,
if there is any.

In [ ]:
if os.path.isfile(f'{save_dir}/pipeline_run_info.yml'):
    with open(f'{save_dir}/pipeline_run_info.yml', "r") as file:
        data = file.read()
    file.close()
    pipeline_run_info = yaml.safe_load(data)
else:
    pipeline_run_info = {}

pipeline_run_info["Chains Used"] = {}
pipeline_run_info["Burn-In"] = {}

Create outputs_and_reports directory

In [ ]:
outputs_and_reports_dir = f'{save_dir}/outputs_and_reports'
os.makedirs(outputs_and_reports_dir)

Set up dictionary of beast_outputs_paths for xml_sets.

In [ ]:
beast_outputs_paths = {}